In [2]:
import skimage.io as io

# Show the figures / plots inside the notebook
%matplotlib inline
from skimage.color import rgb2gray,rgb2hsv, rgba2rgb
import cv2
import matplotlib.pyplot as plt
import numpy as np
from skimage.util import random_noise
import numpy as np
from commonfunctions import *
from skimage import io, color, morphology
from skimage.measure import label, regionprops
from skimage.draw import polygon
from skimage import transform
from skimage.morphology import binary_closing
from scipy import ndimage
from skimage.morphology import remove_small_objects


from skimage.exposure import histogram
from matplotlib.pyplot import bar

In [3]:
pip install opencv-python


Note: you may need to restart the kernel to use updated packages.


In [4]:
import cv2
import numpy as np
import os
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans



In [5]:
import cv2
import numpy as np
from matplotlib import pyplot as plt
from sklearn.cluster import KMeans
from sklearn.neural_network import MLPClassifier
import pandas as pd
import os

# --------------------- Utilities ---------------------
def load_image(path):
    img = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
    if img is None:
        raise ValueError("Image not found")
    return img

def image_crop(img, center_x, center_y, size=256):
    half = size // 2
    x1 = center_x - half
    x2 = center_x + half
    y1 = center_y - half
    y2 = center_y + half
    
    img_h, img_w = img.shape
    pad_top = abs(y1) if y1 < 0 else 0
    pad_bottom = (y2 - img_h) if y2 > img_h else 0
    pad_left = abs(x1) if x1 < 0 else 0
    pad_right = (x2 - img_w) if x2 > img_w else 0
    
    crop_y1 = max(0, y1)
    crop_y2 = min(img_h, y2)
    crop_x1 = max(0, x1)
    crop_x2 = min(img_w, x2)
    
    cropped = img[crop_y1:crop_y2, crop_x1:crop_x2]
    
    if pad_top > 0 or pad_bottom > 0 or pad_left > 0 or pad_right > 0:
        cropped = cv2.copyMakeBorder(cropped, pad_top, pad_bottom, 
                                    pad_left, pad_right, 
                                    cv2.BORDER_CONSTANT, value=0)
    return cropped

# --------------------- Preprocessing (Paper Method) ---------------------
def tophat_transform(img, se_size=3):
    """Top-Hat transform as described in paper"""
    se = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (se_size, se_size))
    # Opening operation
    opened = cv2.morphologyEx(img, cv2.MORPH_OPEN, se)
    # Top-Hat = Original - Opening
    tophat = cv2.subtract(img, opened)
    return tophat

def extract_window_features(img, window_size=5):
    """Extract mean and std features using 5x5 window (paper method)"""
    kernel = np.ones((window_size, window_size), np.float32) / (window_size * window_size)
    
    # Mean
    mean_img = cv2.filter2D(img.astype(np.float32), -1, kernel)
    
    # Standard deviation
    sqr_img = (img.astype(np.float32)) ** 2
    mean_sqr = cv2.filter2D(sqr_img, -1, kernel)
    std_img = np.sqrt(np.maximum(mean_sqr - mean_img**2, 0))
    
    return mean_img, std_img

# --------------------- Feature Vector Construction (Paper Method) ---------------------
def build_feature_vectors(original, enhanced, mean, std):
    """Build 4-dimensional feature vectors as in paper"""
    h, w = original.shape
    n_pixels = h * w
    
    FV = np.zeros((n_pixels, 4), dtype=np.float32)
    FV[:, 0] = original.flatten()
    FV[:, 1] = enhanced.flatten()
    FV[:, 2] = mean.flatten()
    FV[:, 3] = std.flatten()
    
    return FV

# --------------------- Clustering (Paper Method) ---------------------
def segment_with_kmeans(FV, n_clusters=2):
    """K-means segmentation as described in paper"""
    kmeans = KMeans(n_clusters=n_clusters, init='random', 
                   max_iter=100, n_init=10, random_state=42)
    labels = kmeans.fit_predict(FV)
    centers = kmeans.cluster_centers_
    return labels, centers

def identify_mc_cluster(labels, centers, FV, enhanced_flat):
    """
    Identify MC cluster using paper's criteria:
    - Minimum number of data with maximum gray level
    - Cluster separability
    """
    n_clusters = len(centers)
    cluster_scores = []
    
    for i in range(n_clusters):
        cluster_mask = (labels == i)
        cluster_size = np.sum(cluster_mask)
        
        # Mean intensity of cluster in enhanced image
        cluster_intensity = np.mean(enhanced_flat[cluster_mask])
        
        # Prefer smaller clusters with higher intensity (MCs are bright spots)
        # Score favors high intensity and penalizes large size
        score = cluster_intensity / (np.log(cluster_size + 2))
        cluster_scores.append((i, score, cluster_size, cluster_intensity))
    
    # Select cluster with highest score
    cluster_scores.sort(key=lambda x: x[1], reverse=True)
    mc_cluster_id = cluster_scores[0][0]
    
    return mc_cluster_id

# --------------------- Ground Truth Generation ---------------------
def create_ground_truth_mask(shape, center_x, center_y, radius):
    """Create circular ground truth mask"""
    mask = np.zeros(shape, dtype=np.uint8)
    h, w = shape
    
    Y, X = np.ogrid[:h, :w]
    dist_sq = (X - center_x)**2 + (Y - center_y)**2
    mask[dist_sq <= radius**2] = 1
    
    return mask

# --------------------- Evaluation (Paper Method) ---------------------
def evaluate_pixel_classification(pred_labels, gt_mask, mc_cluster_id, shape):
    """
    Evaluate as in paper: pixel-level classification
    - Sensitivity: TP / (TP + FN) 
    - Specificity: TN / (TN + FP)
    - Accuracy: (TP + TN) / Total
    """
    pred_mask = (pred_labels == mc_cluster_id).astype(np.uint8).reshape(shape)
    gt_flat = gt_mask.flatten()
    pred_flat = pred_mask.flatten()
    
    TP = np.sum((pred_flat == 1) & (gt_flat == 1))
    TN = np.sum((pred_flat == 0) & (gt_flat == 0))
    FP = np.sum((pred_flat == 1) & (gt_flat == 0))
    FN = np.sum((pred_flat == 0) & (gt_flat == 1))
    
    total = TP + TN + FP + FN
    
    sensitivity = TP / (TP + FN + 1e-10) * 100
    specificity = TN / (TN + FP + 1e-10) * 100
    accuracy = (TP + TN) / (total + 1e-10) * 100
    
    return {
        'accuracy': accuracy,
        'sensitivity': sensitivity,
        'specificity': specificity,
        'TP': TP,
        'TN': TN,
        'FP': FP,
        'FN': FN
    }

# --------------------- Main Processing Pipeline ---------------------
def process_roi_paper_method(roi, gt_center_x, gt_center_y, gt_radius):
    """
    Complete processing pipeline following paper's methodology
    """
    # Step 1: Top-Hat Enhancement
    enhanced = tophat_transform(roi, se_size=3)
    
    # Step 2: Window-based feature extraction
    mean_img, std_img = extract_window_features(enhanced, window_size=5)
    
    # Step 3: Build feature vectors
    FV = build_feature_vectors(roi, enhanced, mean_img, std_img)
    
    # Step 4: K-means clustering
    labels, centers = segment_with_kmeans(FV, n_clusters=2)
    
    # Step 5: Identify MC cluster
    mc_cluster_id = identify_mc_cluster(labels, centers, FV, enhanced.flatten())
    
    # Step 6: Create ground truth
    gt_mask = create_ground_truth_mask(roi.shape, gt_center_x, gt_center_y, gt_radius)
    
    # Step 7: Evaluate
    metrics = evaluate_pixel_classification(labels, gt_mask, mc_cluster_id, roi.shape)
    
    # For visualization
    pred_mask = (labels == mc_cluster_id).astype(np.uint8).reshape(roi.shape)
    
    return enhanced, pred_mask, gt_mask, metrics

# --------------------- Batch Processing ---------------------
def process_dataset(csv_path, visualize_first=True):
    """Process entire dataset"""
    df = pd.read_csv(csv_path)
    
    # Filter for CALC cases with valid coordinates
    calc_cases = df[(df['CLASS'] == 'CALC') & (df['X'].notna())].copy()
    print(f"Found {len(calc_cases)} Microcalcification cases\n")
    
    results = []
    
    print(f"{'Image':<12} | {'Acc%':<8} | {'Sens%':<8} | {'Spec%':<8} | Status")
    print("-" * 60)
    
    for idx, (index, row) in enumerate(calc_cases.iterrows()):
        ref_num = row['REF']
        image_path = row['PATH']
        
        try:
            raw_x = int(row['X'])
            raw_y = int(row['Y'])
            radius = int(row['RADIUS'])
        except (ValueError, KeyError):
            print(f"{ref_num:<12} | SKIP: Invalid data")
            continue
        
        # Handle Y-flip for MIAS
        center_y = 1024 - raw_y
        center_x = raw_x
        
        
        if not os.path.exists(image_path):
            print(f"{ref_num:<12} | ERROR: File not found")
            continue
            
            # Load and crop
        img = load_image(image_path)
        roi = image_crop(img, center_x, center_y, size=256)
            
            # ROI center is always at 128, 128
        roi_center = 128
            
            # Process using paper method
        enhanced, pred_mask, gt_mask, metrics = process_roi_paper_method(
            roi, roi_center, roi_center, radius
        )
            
        results.append({
            'ref': ref_num,
            'accuracy': metrics['accuracy'],
            'sensitivity': metrics['sensitivity'],
            'specificity': metrics['specificity']
        })
            
        status = "✓" if metrics['sensitivity'] > 50 else "✗"
        print(f"{ref_num:<12} | {metrics['accuracy']:>6.2f} | "
                f"{metrics['sensitivity']:>6.2f} | {metrics['specificity']:>6.2f} | {status}")
            
           
            
        
    
    # Summary
    print("\n" + "="*60)
    if results:
        avg_acc = np.mean([r['accuracy'] for r in results])
        avg_sens = np.mean([r['sensitivity'] for r in results])
        avg_spec = np.mean([r['specificity'] for r in results])
        
        print(f"SUMMARY ({len(results)} images processed):")
        print(f"Average Accuracy:    {avg_acc:.2f}%")
        print(f"Average Sensitivity: {avg_sens:.2f}%")
        print(f"Average Specificity: {avg_spec:.2f}%")
    else:
        print("No images processed successfully")
    
    return results

# --------------------- Run ---------------------
if __name__ == "__main__":
    CSV_PATH = "train_dataset.csv"
    results = process_dataset(CSV_PATH, visualize_first=True)

Found 22 Microcalcification cases

Image        | Acc%     | Sens%    | Spec%    | Status
------------------------------------------------------------
mdb240       |  40.68 | 100.00 |  39.15 | ✓
mdb231       |  52.44 |  40.97 |  53.61 | ✗
mdb239       |  14.70 | 100.00 |   7.61 | ✓
mdb223       |  20.48 | 100.00 |  20.35 | ✓
mdb256       |  73.74 |  61.84 |  74.57 | ✓
mdb249       |  26.79 | 100.00 |  17.74 | ✓
mdb219       |  44.81 |  98.25 |  42.58 | ✓
mdb211       |  16.04 | 100.00 |  15.36 | ✓
mdb214       |  74.53 |   0.00 |  74.96 | ✗
mdb238       |  26.97 | 100.00 |  25.95 | ✓
mdb239       |  38.52 | 100.00 |  36.62 | ✓
mdb241       |  46.27 |  70.62 |  44.47 | ✓
mdb252       |  89.38 |   0.00 |  91.69 | ✗
mdb253       |  12.75 | 100.00 |   9.35 | ✓
mdb249       |  43.83 |  86.03 |  33.54 | ✓
mdb226       |  38.12 | 100.00 |  37.98 | ✓
mdb248       |  55.67 | 100.00 |  55.46 | ✓
mdb218       |  23.68 | 100.00 |  23.45 | ✓
mdb226       |  24.48 | 100.00 |  22.15 | ✓
mdb222       

In [6]:
def get_mc_data(roi):
    """
    Lightweight getter for MC extraction.
    Uses existing functions and keeps logic minimal.
    """

    if roi is None or roi.size == 0:
        return [], None

    # 1. Use existing enhancement
    tophat_norm = tophat_transform(roi)

    # 2. Simple binary map for MC candidates
    _, bw = cv2.threshold(
        tophat_norm,
        0, 255,
        cv2.THRESH_BINARY + cv2.THRESH_OTSU
    )

    # 3. Find contours (no extra processing)
    contours, _ = cv2.findContours(
        bw,
        cv2.RETR_EXTERNAL,
        cv2.CHAIN_APPROX_SIMPLE
    )

    return contours, tophat_norm
